In [1]:
import pandas as pd

In [2]:
# 1. Importar datos desde un archivo Excel
# Cambia 'ruta_al_archivo.xlsx' por la ruta real de tu archivo
ruta_archivo = "ATENEO BASE PLANA (3).xlsx"
df = pd.read_excel(ruta_archivo, header=3)

In [3]:
for col in df.select_dtypes(include=["object", "string"]).columns:
    df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)

In [4]:
# Reemplaza la llave por el valor en el diccionario reemplazos. Ejemplo: 28 por 30 y 56 por 60
reemplazos = {28: 30, 56: 60}

df['dosis_aux'] = df['Dosis3'].replace(reemplazos)

In [5]:
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)

In [6]:
df_filter_tipo = df[df["TIPO"].isin(["ME", "MINV"])]

In [7]:
# 3. Filtrar los datos del laboratorio "Siegfried"
df_siegfried = df_filter_tipo[df_filter_tipo["Laboratorio"].str.strip() == "Siegfried"]
df_siegfried = df_siegfried[df_siegfried["TIPO"].isin(["ME", "MINV"])]
df_siegfried = df_siegfried.drop_duplicates()
df_siegfried.reset_index(drop=True, inplace=True)
df_siegfried.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 98 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TIPO                         833 non-null    object 
 1   TIPO-MERCADO                 833 non-null    object 
 2   CODIGO                       833 non-null    object 
 3   CT                           833 non-null    object 
 4   CT4                          833 non-null    object 
 5   Molécula(s)                  833 non-null    object 
 6   Corp                         833 non-null    object 
 7   Xplora                       833 non-null    object 
 8   Laboratorio                  833 non-null    object 
 9   Marca                        833 non-null    object 
 10  Presentación                 833 non-null    object 
 11  FF                           833 non-null    object 
 12  Lab - Marca                  833 non-null    object 
 13  Lab - Presentacion  

In [8]:
# Elimina columnas de la 22 a la 88 (índices basados en 0)
df_filter_tipo = df_filter_tipo.drop(df_filter_tipo.iloc[:, 22:89], axis=1)

In [9]:
# Ahora hacemos merge sobre el df completo
df_completo_type_dosis_mol = df_filter_tipo.merge(
    df_siegfried[
        [
            "TIPO",
            "FF",
            "Concentration Type & Factor",
            "dosis_aux",
            "Molécula(s)",
            "Precio PVP",
        ]
        
    ],
    left_on=["TIPO", "FF", "dosis_aux", "Concentration Type & Factor", "Molécula(s)"],
    right_on=["TIPO", "FF", "dosis_aux", "Concentration Type & Factor", "Molécula(s)"],
    how="left",
    suffixes=("", "_Siegfried"),
)


In [10]:
df_completo_type_dosis_mol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22694 entries, 0 to 22693
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TIPO                         22694 non-null  object 
 1   TIPO-MERCADO                 22694 non-null  object 
 2   CODIGO                       22694 non-null  object 
 3   CT                           22692 non-null  object 
 4   CT4                          22694 non-null  object 
 5   Molécula(s)                  22694 non-null  object 
 6   Corp                         22694 non-null  object 
 7   Xplora                       22694 non-null  object 
 8   Laboratorio                  22694 non-null  object 
 9   Marca                        22694 non-null  object 
 10  Presentación                 22694 non-null  object 
 11  FF                           22688 non-null  object 
 12  Lab - Marca                  22694 non-null  object 
 13  Lab - Presentaci

In [11]:
# Renombrar la columna
df_completo_type_dosis_mol.rename(columns={"Precio PVP_Siegfried": "Precio_Siegfried"}, inplace=True)

# Corregir la condición de la asignación
df_completo_type_dosis_mol.loc[
    df_completo_type_dosis_mol["Laboratorio"] == "Siegfried", "Precio_Siegfried"
] = None

In [12]:
df_completo_type_dosis_mol['dif_vs_sieg'] = (
    df_completo_type_dosis_mol['Precio_Siegfried'] / df_completo_type_dosis_mol['Precio PVP'] - 1
)

In [13]:
print(df_completo_type_dosis_mol[df_completo_type_dosis_mol['dif_vs_sieg'].notna()][['Precio_Siegfried', 'Precio PVP', 'dif_vs_sieg']].head())

    Precio_Siegfried  Precio PVP  dif_vs_sieg
26          19093.08    20555.00    -0.071122
27          13512.67    20555.00    -0.342609
28          19093.08    20555.00    -0.071122
29          13512.67    20555.00    -0.342609
36           8704.98     9974.25    -0.127255


In [14]:
df_completo_type_dosis_mol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22694 entries, 0 to 22693
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TIPO                         22694 non-null  object 
 1   TIPO-MERCADO                 22694 non-null  object 
 2   CODIGO                       22694 non-null  object 
 3   CT                           22692 non-null  object 
 4   CT4                          22694 non-null  object 
 5   Molécula(s)                  22694 non-null  object 
 6   Corp                         22694 non-null  object 
 7   Xplora                       22694 non-null  object 
 8   Laboratorio                  22694 non-null  object 
 9   Marca                        22694 non-null  object 
 10  Presentación                 22694 non-null  object 
 11  FF                           22688 non-null  object 
 12  Lab - Marca                  22694 non-null  object 
 13  Lab - Presentaci

In [15]:
df_completo_type_dosis_mol = df_completo_type_dosis_mol.drop_duplicates()
df_completo_type_dosis_mol.reset_index(drop=True, inplace=True)


In [16]:
df_completo_type_dosis_mol

,TIPO,TIPO-MERCADO,CODIGO,CT,CT4,Molécula(s),Corp,Xplora,Laboratorio,Marca,...,Concentration Type & Factor,manual farmaceutico,Dosis3,Precio PVP,Fecha version,duplicados,ATC IV,dosis_aux,Precio_Siegfried,dif_vs_sieg
0,MINV,E,0002702,C09D1 - Comb Antg At2 C2 Y/o Diu,C - Aparato Cardiovascular,Candersatan + Hidroclorotiazida,Abbott Corp,Abbott Epd,Abbott Epd,Tiadyl Plus,...,0.00 ??,29638,30.0,89020.07,16/01/2025,2,C09D1,30.0,NaN,NaN
1,MINV,E,0004101,C07A0 - Agentes Beta-bloq.solos,C - Aparato Cardiovascular,Atenolol,Klonal,Klonal,Klonal,Atenoblock,...,50.0 MG,19789,30.0,3531.98,16/01/2025,1,C07A0,30.0,NaN,NaN
2,MINV,E,0006201,N06A4 - Inh.selec.recapt.seroton,N - Sistema Nervioso,Paroxetina,Baliarda Corp,Baliarda,Baliarda,Meplar,...,20.0 MG,28816,10.0,7917.11,16/01/2025,1,C07A0,10.0,NaN,NaN
3,MINV,E,0006202,N06A4 - Inh.selec.recapt.seroton,N - Sistema Nervioso,Paroxetina,Baliarda Corp,Baliarda,Baliarda,Meplar,...,20.0 MG,29376,30.0,41765.06,16/01/2025,1,N06A4,30.0,NaN,NaN
4,MINV,E,0006204,N06A4 - Inh.selec.recapt.seroton,N - Sistema Nervioso,Paroxetina,Baliarda Corp,Baliarda,Baliarda,Meplar,...,30.0 MG,43481,30.0,45097.01,16/01/2025,1,N06A4,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20365,MINV,E,9029103,J01D1 - Cefalosporinas Orales,J - Antiinfecciosos Via Gene,Cefalexina,Klonal,Klonal,Klonal,Velexina,...,500 MG,31713,1.0,9279.46,16/01/2025,250501,R05C250499,1.0,12602.04,0.358057
20366,MINV,E,9029150,J01D1 - Cefalosporinas Orales,J - Antiinfecciosos Via Gene,Cefalexina,Klonal,Klonal,Klonal,Velexina,...,500 MG,19861,8.0,NaN,16/01/2025,250502,R05C250500,8.0,NaN,NaN
20367,MINV,E,9029155,J01D1 - Cefalosporinas Orales,J - Antiinfecciosos Via Gene,Cefalexina,Klonal,Klonal,Klonal,Velexina,...,500 MG,19862,16.0,7771.21,16/01/2025,250503,R05C250501,16.0,NaN,NaN
20368,MINV,E,9355733,J01F0 - Macrolidos Y Similares,J - Antiinfecciosos Via Gene,Azitromicina,Pfizer Corp,Pfizer,Pfizer,Zitromax,...,600 MG,12629,1.0,19465.62,16/01/2025,250504,R05C250502,1.0,NaN,NaN


In [17]:
df_completo_type_dosis_mol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20370 entries, 0 to 20369
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TIPO                         20370 non-null  object 
 1   TIPO-MERCADO                 20370 non-null  object 
 2   CODIGO                       20370 non-null  object 
 3   CT                           20368 non-null  object 
 4   CT4                          20370 non-null  object 
 5   Molécula(s)                  20370 non-null  object 
 6   Corp                         20370 non-null  object 
 7   Xplora                       20370 non-null  object 
 8   Laboratorio                  20370 non-null  object 
 9   Marca                        20370 non-null  object 
 10  Presentación                 20370 non-null  object 
 11  FF                           20364 non-null  object 
 12  Lab - Marca                  20370 non-null  object 
 13  Lab - Presentaci

In [18]:
df_completo_type_dosis_mol.to_excel("df_completo_type_dosis_mol_ff.xlsx", index=False)